<font color='red'> Penalty kick application based on 'Professionals Play Minimax' by Ignacio Palacios-Huerta (2003). </font>

### Imports

In [ ]:
from bokeh.io import show, output_notebook, save, output_file
from bokeh.models import (Plot, TapTool, ColumnDataSource, LabelSet, 
                          StaticLayoutProvider, Circle, MultiLine)
from bokeh.models.widgets import Div
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.plotting import figure
from bokeh.models.renderers import GraphRenderer, GlyphRenderer
from bokeh.layouts import layout, row, column, gridplot
from bokeh.models import (CustomJSHover, CustomJS, CustomJSTransform, Button, Slider, DataTable, 
                          ColumnDataSource, TableColumn, Rect, Dropdown, 
                          Legend, LegendItem, Circle, HoverTool)
from bokeh.models.glyphs import Text
from bokeh.events import ButtonClick
from bokeh.transform import transform
import main_game_figure
import game_stats_figure_1
import game_stats_figure_2
import game_stats_figure_3
import penalty_kick_automated_game
#TODO: IMPROVE THE CODE STRUCTURE, MAKE OPTIMIZATIONS. CODE IN THE JUPYTER NOTEBOOK 
#      SHOULD ONLY BE CODE THAT THE USER NEEDS TO SEE TO UNDERSTAND HOW THE DEMO WORKS.

In [ ]:
output_notebook()

### Define Game Figure and Sprites

In [ ]:
#Set Up Penalty Kick Game Figure
#HAS MANY PARAMETERS TO CONFIGURE THE FIGURE, CHECK THE SCRIPT TO CHECK THEM. 
game_figure, goalie_head, goalie_body, ball = main_game_figure.game_figure_setup() 

### Define Game Stats Figure 1 and Sprites

In [ ]:
(game_stats_figure_1, game_stats_figure_1_source, ll_scored_bar, lm_scored_bar, lr_scored_bar, rl_scored_bar, 
 rm_scored_bar, rr_scored_bar, ll_blocked_left_bar, lm_blocked_left_bar, lr_blocked_left_bar, rl_blocked_left_bar,
 rm_blocked_left_bar, rr_blocked_left_bar, ll_blocked_middle_bar, lm_blocked_middle_bar,lr_blocked_middle_bar, 
 rl_blocked_middle_bar, rm_blocked_middle_bar, rr_blocked_middle_bar, ll_blocked_right_bar, lm_blocked_right_bar,
 lr_blocked_right_bar, rl_blocked_right_bar, rm_blocked_right_bar, rr_blocked_right_bar
 ) = game_stats_figure_1.stats_figure_1_setup()

### Define Game Stats Figure 2

In [ ]:
(game_stats_figure_2, game_stats_figure_2_source) = game_stats_figure_2.stats_figure_2_setup()

### Define Game Stats Figure 3

In [ ]:
(game_stats_figure_3, game_stats_figure_3_source) = game_stats_figure_3.stats_figure_3_setup()

### Define Labels

In [ ]:
scr_text = ColumnDataSource({'x' : [2, 70, 2, 14, 14],
                             'y' : [86, 86, 5, 40, 32],
                             'text' : ['Rounds played: 0',
                                      'Total score: 0',
                                      '',
                                      '',
                                      '']})

labels = Text(x = "x", y = "y", text = 'text', text_color = "whitesmoke",
              text_font_size = '15pt', x_offset = 0, y_offset = +9,
              text_baseline = 'ideographic', text_align = 'left')

game_figure.add_glyph(scr_text, labels);

### Define State Variables as Divs

In [ ]:
nround = Div(text = '0', visible = False) # total number of rounds completed
score = Div(text = '0', visible = False) # current score
freq_left_foot = Div(text = '0,0,0', visible = False) # frequency of left, middle, and right kicks (left-footed)
freq_right_foot = Div(text = '0,0,0', visible = False) # frequency of left, middle, and right kicks (right-footed)
kicker_foot = Div(text = '', visible = False) # current footedness of the kicker
kicker_kick = Div(text = '', visible = False) # current direction the kicker will kick

(iterations_to_run, strategy_to_use,
 ll_scored, lm_scored, lr_scored, rl_scored, rm_scored, rr_scored,
 ll_blocked_left, lm_blocked_left, lr_blocked_left,
 rl_blocked_left, rm_blocked_left, rr_blocked_left,
 ll_blocked_middle, lm_blocked_middle, lr_blocked_middle,
 rl_blocked_middle, rm_blocked_middle, rr_blocked_middle,
 ll_blocked_right, lm_blocked_right, lr_blocked_right,
 rl_blocked_right, rm_blocked_right, rr_blocked_right) = penalty_kick_automated_game.create_gamestate_divs()

### Define Buttons

In [ ]:
#Start and Next Buttons:
b_start = Button(label = "Begin", button_type = "success", 
                 sizing_mode = 'scale_width', width_policy = 'fit')
b_next = Button(label = "Next round", button_type = "success", 
                sizing_mode = 'scale_width', width_policy = 'fit', disabled = True)

#Right and Left Footed Buttons:
b_fl = Button(label = "Left-Footed", button_type = "success", 
              sizing_mode = 'scale_width', width_policy = 'fit', disabled = True)
b_fr = Button(label = "Right-Footed", button_type = "success", 
              sizing_mode = 'scale_width', width_policy = 'fit', disabled = True)

#Right, Middle, and Left Kick Buttons:
bl = Button(label = "Left", button_type = "success", sizing_mode = 'scale_width', 
            width_policy = 'fit', disabled = False, visible = False)
bm = Button(label = "Middle", button_type = "success", sizing_mode = 'scale_width', 
            width_policy = 'fit', disabled = False, visible = False)
br = Button(label = "Right", button_type = "success", sizing_mode = 'scale_width', 
            width_policy = 'fit', disabled = False, visible = False)

#Shoot Button:
b_shoot = Button(label = "SHOOT!", button_type = "success", sizing_mode = 'scale_width', 
                 width_policy = 'fit', disabled = False, visible = False)

#Automate Buttons:
(b_automate, b_start_automate, b_auto_next) = penalty_kick_automated_game.create_buttons()

### Define Sliders

In [ ]:
(LL_aim_slider, LM_aim_slider, LR_aim_slider,
 RL_aim_slider, RM_aim_slider, RR_aim_slider,
 iterations_slider) = penalty_kick_automated_game.create_sliders()

### Define Dropdown Menu

In [ ]:
strategy_dropdown = penalty_kick_automated_game.create_strategy_dropdown()

### Define Automation Distribution Tracking Table

In [ ]:
automation_distribution_table_source = penalty_kick_automated_game.create_distribution_table_source()
automation_distribution_table = penalty_kick_automated_game.create_distribution_table(automation_distribution_table_source)

### Define Automation Table

In [ ]:
automation_table_source = penalty_kick_automated_game.create_automation_table_source()
automation_table = penalty_kick_automated_game.create_automation_table(automation_table_source)

### Define Callbacks

In [ ]:
#Next Button:
b_next_code = """
b_shoot.visible = false;

b_fr.visible = true;
b_fl.visible = true;
b_fr.disabled = false;
b_fl.disabled = false;

bl.visible = false;
bm.visible = false;
br.visible = false;

b_next.disabled = true;

txt.data['text'][2] = '';
txt.data['text'][3] = 'Choose a right- or left footed kicker!';
txt.change.emit();

ball.x = 50;
ball.y = 13;
goalie_head.x = 50;
goalie_body.x = 50;
"""

b_next_click = CustomJS(args = dict(b_start = b_start, b_fl = b_fl, b_fr = b_fr, bl = bl, bm = bm, br = br,
                                    txt = scr_text, b_shoot = b_shoot, b_next = b_next, ball = ball,
                                    goalie_head = goalie_head, goalie_body = goalie_body),
                        code = b_next_code)

b_next.js_on_click(b_next_click)

In [ ]:
#Start Button:
b_start_code = b_next_code + """
if (b_start.label == 'Begin'){
    b_start.label = 'Restart';
}
else{
    txt.data['text'][0] = 'Rounds played: 0';
    txt.data['text'][1] = 'Total score: 0';
    txt.data['text'][2] = '';
    txt.data['text'][3] = '';
    txt.data['text'][4] = '';
    nround.text = '0';
    score.text = '0';
    freq_left_foot.text = '0,0,0';
    freq_right_foot.text = '0,0,0';
    kicker_foot.text = '';
    kicker_kick.text = '';
    txt.change.emit();
    b_start.label = 'Begin';
}
"""

b_start_click = CustomJS(args = dict(b_start = b_start, b_fl = b_fl, b_fr = b_fr, bl = bl, bm = bm, br = br,
                                     txt = scr_text, b_shoot=b_shoot, b_next = b_next, ball = ball,
                                     goalie_head = goalie_head, goalie_body = goalie_body, nround=nround,
                                     score = score, freq_left_foot = freq_left_foot, 
                                     freq_right_foot = freq_right_foot, kicker_foot = kicker_foot, 
                                     kicker_kick = kicker_kick),
                         code = b_start_code)

b_start.js_on_click(b_start_click)

In [ ]:
#Create the Striker Foot Buttons:

#Select the Foot Type of the Striker Buttons:
def b_f_click(foot):
    code = """
    b_fl.disabled = true;
    b_fr.disabled = true;
    b_fl.visible = false;
    b_fr.visible = false;
    bl.visible = true;
    bm.visible = true;
    br.visible = true;
    txt.data['text'][3] = 'Choose where to kick!';
    txt.data['text'][2] = '""" + foot + """-footed kicker.';
    kicker_foot.text = '""" + foot + """';
    txt.change.emit();
    """
    return CustomJS(args = dict(b_fl = b_fl, b_fr = b_fr, bl = bl, bm = bm, br = br,
                                txt = scr_text, kicker_foot = kicker_foot),
                    code = code)

b_fl.js_on_click(b_f_click('Left'))
b_fr.js_on_click(b_f_click('Right'))

In [ ]:
#Create the Kicker Kick Buttons:

#Select the Kick Direction of the Kicker Kick Buttons:
def b_kick_click(kick):
    code = """
    b_shoot.visible = true;
    bl.visible = false;
    bm.visible = false;
    br.visible = false;
    txt.data['text'][3] = '';
    var current = txt.data['text'][2];
    txt.data['text'][2] = current + ' Kicking """+kick+""".';
    kicker_kick.text= '"""+kick+"""';
    txt.change.emit();
    b_shoot.disabled = false;
    """
    return CustomJS(args = dict(b_shoot = b_shoot, bl = bl, bm = bm, br = br,
                                txt = scr_text, kicker_kick = kicker_kick),
                    code = code)

bl.js_on_click(b_kick_click('Left'))
bm.js_on_click(b_kick_click('Middle'))
br.js_on_click(b_kick_click('Right'))

In [ ]:
# shoot button
b_shoot_code = """
// Define probability matrix
var p = {'Right' : {'LeftLeft' : 0.55,
                    'LeftMiddle' : 0.65,
                    'LeftRight' : 0.93,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.95,
                    'RightMiddle' : 0.73,
                    'RightRight' : 0.70},
         'Left' :  {'LeftLeft' : 0.67,
                    'LeftMiddle' : 0.70,
                    'LeftRight' : 0.96,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.87,
                    'RightMiddle' : 0.65,
                    'RightRight' : 0.61}};

// Choose best action
var freq;

if (kicker_foot.text == 'Right'){
    freq = freq_right_foot.text.split(',').map(Number);
}
else{
    freq = freq_left_foot.text.split(',').map(Number);
}

var kicker_action = 'Left';
var expected = (freq[0] * p[kicker_foot.text]['LeftLeft']
                + freq[1]*p[kicker_foot.text]['MiddleLeft']
                + freq[2]*p[kicker_foot.text]['RightLeft']);
var actions = ["Left", "Middle", "Right"];

for (var i = 0; i < 3; i++){
    var val = (freq[0] * p[kicker_foot.text]['Left'+actions[i]]
               + freq[1]*p[kicker_foot.text]['Middle'+actions[i]]
               + freq[2]*p[kicker_foot.text]['Right'+actions[i]]);
    if (val < expected){
        kicker_action = actions[i];
        expected = val;
    }
}

// Determine if goal
var goal = 1;
if (Math.random() > p[kicker_foot.text][kicker_kick.text + kicker_action]){
    goal = -1;
}

// Animate ball and goalie
ball.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_kick.text];
ball.y = 63;
goalie_head.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_action];
goalie_body.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_action];

// Add to frequency history
var dict = {'Left' : 0,
            'Middle' : 1,
            'Right' : 2};
if (kicker_foot.text == 'Right'){
    var freq = freq_right_foot.text.split(',');
    freq[dict[kicker_kick.text]] = parseInt(freq[dict[kicker_kick.text]]) + 1;
    freq_right_foot.text = freq.toString();
}
else{
    var freq = freq_left_foot.text.split(',');
    freq[dict[kicker_kick.text]] = parseInt(freq[dict[kicker_kick.text]]) + 1;
    freq_left_foot.text = freq.toString();
}

// Update text
var n = (parseInt(nround.text) + 1);
nround.text = n.toString();
txt.data['text'][0] = 'Rounds played: ' + n;

if (goal == 1){
    txt.data['text'][3] = 'GOAL!';
}
else{
    txt.data['text'][3] = 'Blocked';
}

var s = parseInt(score.text) + goal;
score.text = s.toString();
txt.data['text'][1] = 'Total score: ' + s;

txt.change.emit();

// Allow starting next round
b_shoot.disabled = true;
b_next.disabled = false;
"""

b_shoot_click = CustomJS(args = dict(kicker_foot = kicker_foot, freq_right_foot = freq_right_foot,
                                     freq_left_foot = freq_left_foot, kicker_kick = kicker_kick,
                                     ball = ball, goalie_head = goalie_head, goalie_body = goalie_body,
                                     nround = nround, txt = scr_text, score = score, b_shoot = b_shoot,
                                     b_next = b_next),
                         code = b_shoot_code)

b_shoot.js_on_click(b_shoot_click)

In [ ]:
args_dict = dict(b_automate = b_automate, b_start = b_start, b_fl = b_fl, b_fr = b_fr, 
                 b_next = b_next, LL_aim_slider = LL_aim_slider, LM_aim_slider = LM_aim_slider,
                 LR_aim_slider = LR_aim_slider, RL_aim_slider = RL_aim_slider,
                 RM_aim_slider = RM_aim_slider, RR_aim_slider = RR_aim_slider,
                 iterations_slider = iterations_slider, strategy_dropdown = strategy_dropdown,
                 automation_table = automation_table, txt = scr_text)
penalty_kick_automated_game.b_automate_setup(b_automate = b_automate, args_dict = args_dict)

In [ ]:
args_dict = dict(b_start_automate = b_start_automate, b_auto_next = b_auto_next,
                 LL_aim_slider = LL_aim_slider, LM_aim_slider = LM_aim_slider,
                 LR_aim_slider = LR_aim_slider, RL_aim_slider = RL_aim_slider,
                 RM_aim_slider = RM_aim_slider, RR_aim_slider = RR_aim_slider,
                 iterations_slider = iterations_slider, strategy_dropdown = strategy_dropdown,
                 automation_table = automation_table,
                 automation_distribution_table = automation_distribution_table,
                 ChancesColumnDataSource = automation_table_source,
                 DistributionColumnDataSource = automation_distribution_table_source,
                 txt = scr_text, strategy_to_use = strategy_to_use, nround = nround,
                 iterations_to_run = iterations_to_run, game_figure = game_figure,
                 game_stats_figure_1 = game_stats_figure_1, 
                 game_stats_figure_2 = game_stats_figure_2, 
                 game_stats_figure_3 = game_stats_figure_3, score = score,
                 ball = ball, goalie_head = goalie_head, goalie_body = goalie_body,
                 game_stats_figure_1_source = game_stats_figure_1_source,
                 ll_scored_bar = ll_scored_bar, lm_scored_bar = lm_scored_bar,
                 lr_scored_bar = lr_scored_bar, rl_scored_bar = rl_scored_bar,
                 rm_scored_bar = rm_scored_bar, rr_scored_bar = rr_scored_bar,
                 ll_scored = ll_scored, lm_scored = lm_scored,
                 lr_scored = lr_scored, rl_scored = rl_scored,
                 rm_scored = rm_scored, rr_scored = rr_scored,
                 ll_blocked_left_bar = ll_blocked_left_bar,
                 lm_blocked_left_bar = lm_blocked_left_bar,
                 lr_blocked_left_bar = lr_blocked_left_bar,
                 rl_blocked_left_bar = rl_blocked_left_bar,
                 rm_blocked_left_bar = rm_blocked_left_bar,
                 rr_blocked_left_bar = rr_blocked_left_bar,
                 ll_blocked_left = ll_blocked_left, lm_blocked_left = lm_blocked_left,
                 lr_blocked_left = lr_blocked_left, rl_blocked_left = rl_blocked_left,
                 rm_blocked_left = rm_blocked_left, rr_blocked_left = rr_blocked_left,
                 ll_blocked_middle_bar = ll_blocked_middle_bar,
                 lm_blocked_middle_bar = lm_blocked_middle_bar,
                 lr_blocked_middle_bar = lr_blocked_middle_bar,
                 rl_blocked_middle_bar = rl_blocked_middle_bar,
                 rm_blocked_middle_bar = rm_blocked_middle_bar,
                 rr_blocked_middle_bar = rr_blocked_middle_bar,
                 ll_blocked_middle = ll_blocked_middle, lm_blocked_middle = lm_blocked_middle,
                 lr_blocked_middle = lr_blocked_middle, rl_blocked_middle = rl_blocked_middle,
                 rm_blocked_middle = rm_blocked_middle, rr_blocked_middle = rr_blocked_middle,
                 ll_blocked_right_bar = ll_blocked_right_bar,
                 lm_blocked_right_bar = lm_blocked_right_bar,
                 lr_blocked_right_bar = lr_blocked_right_bar,
                 rl_blocked_right_bar = rl_blocked_right_bar,
                 rm_blocked_right_bar = rm_blocked_right_bar,
                 rr_blocked_right_bar = rr_blocked_right_bar,
                 ll_blocked_right = ll_blocked_right, lm_blocked_right = lm_blocked_right,
                 lr_blocked_right = lr_blocked_right, rl_blocked_right = rl_blocked_right,
                 rm_blocked_right = rm_blocked_right, rr_blocked_right = rr_blocked_right,
                 game_stats_figure_2_source = game_stats_figure_2_source,
                 game_stats_figure_3_source = game_stats_figure_3_source)

penalty_kick_automated_game.b_start_automate_setup(b_start_automate = b_start_automate, args_dict = args_dict)

In [ ]:
args_dict = dict(ChancesColumnDataSource = automation_table_source,
                 DistributionColumnDataSource = automation_distribution_table_source,
                 txt = scr_text, strategy_to_use = strategy_to_use, nround = nround,
                 iterations_to_run = iterations_to_run, b_auto_next = b_auto_next,
                 game_figure = game_figure, 
                 automation_distribution_table = automation_distribution_table,
                 game_stats_figure_1 = game_stats_figure_1, 
                 game_stats_figure_2 = game_stats_figure_2, 
                 game_stats_figure_3 = game_stats_figure_3, score = score, 
                 ball = ball, goalie_head = goalie_head, goalie_body = goalie_body,
                 game_stats_figure_1_source = game_stats_figure_1_source,
                 ll_scored_bar = ll_scored_bar, lm_scored_bar = lm_scored_bar,
                 lr_scored_bar = lr_scored_bar, rl_scored_bar = rl_scored_bar,
                 rm_scored_bar = rm_scored_bar, rr_scored_bar = rr_scored_bar,
                 ll_scored = ll_scored, lm_scored = lm_scored,
                 lr_scored = lr_scored, rl_scored = rl_scored,
                 rm_scored = rm_scored, rr_scored = rr_scored,
                 ll_blocked_left_bar = ll_blocked_left_bar,
                 lm_blocked_left_bar = lm_blocked_left_bar,
                 lr_blocked_left_bar = lr_blocked_left_bar,
                 rl_blocked_left_bar = rl_blocked_left_bar,
                 rm_blocked_left_bar = rm_blocked_left_bar,
                 rr_blocked_left_bar = rr_blocked_left_bar,
                 ll_blocked_left = ll_blocked_left, lm_blocked_left = lm_blocked_left,
                 lr_blocked_left = lr_blocked_left, rl_blocked_left = rl_blocked_left,
                 rm_blocked_left = rm_blocked_left, rr_blocked_left = rr_blocked_left,
                 ll_blocked_middle_bar = ll_blocked_middle_bar,
                 lm_blocked_middle_bar = lm_blocked_middle_bar,
                 lr_blocked_middle_bar = lr_blocked_middle_bar,
                 rl_blocked_middle_bar = rl_blocked_middle_bar,
                 rm_blocked_middle_bar = rm_blocked_middle_bar,
                 rr_blocked_middle_bar = rr_blocked_middle_bar,
                 ll_blocked_middle = ll_blocked_middle, lm_blocked_middle = lm_blocked_middle,
                 lr_blocked_middle = lr_blocked_middle, rl_blocked_middle = rl_blocked_middle,
                 rm_blocked_middle = rm_blocked_middle, rr_blocked_middle = rr_blocked_middle,
                 ll_blocked_right_bar = ll_blocked_right_bar,
                 lm_blocked_right_bar = lm_blocked_right_bar,
                 lr_blocked_right_bar = lr_blocked_right_bar,
                 rl_blocked_right_bar = rl_blocked_right_bar,
                 rm_blocked_right_bar = rm_blocked_right_bar,
                 rr_blocked_right_bar = rr_blocked_right_bar,
                 ll_blocked_right = ll_blocked_right, lm_blocked_right = lm_blocked_right,
                 lr_blocked_right = lr_blocked_right, rl_blocked_right = rl_blocked_right,
                 rm_blocked_right = rm_blocked_right, rr_blocked_right = rr_blocked_right,
                 game_stats_figure_2_source = game_stats_figure_2_source,
                 game_stats_figure_3_source = game_stats_figure_3_source)
penalty_kick_automated_game.b_auto_next_setup(b_auto_next = b_auto_next, args_dict = args_dict)

In [ ]:
penalty_kick_automated_game.aim_sliders_setup(ll_aim_slider = LL_aim_slider, lm_aim_slider = LM_aim_slider,
                                              lr_aim_slider = LR_aim_slider, rl_aim_slider = RL_aim_slider,
                                              rm_aim_slider = RM_aim_slider, rr_aim_slider = RR_aim_slider,
                                              automation_table_source = automation_table_source)

In [ ]:
args_dict = dict(iterations_to_run = iterations_to_run,
                 game_stats_figure_1 = game_stats_figure_1,
                 game_stats_figure_2 = game_stats_figure_2,
                 game_stats_figure_3 = game_stats_figure_3,
                 game_stats_figure_2_source = game_stats_figure_2_source,
                 game_stats_figure_3_source = game_stats_figure_3_source)
penalty_kick_automated_game.iterations_slider_setup(iterations_slider = iterations_slider,
                                                    args_dict = args_dict)

In [ ]:
args_dict = dict(strategy_dropdown = strategy_dropdown,
                 strategy_to_use = strategy_to_use,
                 b_start_automate = b_start_automate)
penalty_kick_automated_game.strategy_dropdown_setup(strategy_dropdown = strategy_dropdown,
                                                    args_dict = args_dict)

### Format Layout and Show

In [ ]:
buttons_bot = row(b_start, nround, b_fl, b_fr, b_shoot, bl, bm, br,
                  score, freq_left_foot, freq_right_foot, kicker_foot, 
                  kicker_kick, b_next, max_width = 600, sizing_mode = 'stretch_width')
automate_button = row(b_automate, iterations_slider,  b_auto_next, 
                      max_width = 400, sizing_mode = 'stretch_width')
strategy_dropdown_row = row(strategy_dropdown, max_width = 400, sizing_mode = 'stretch_width')
start_automate_row = row(b_start_automate, max_width = 400, sizing_mode = 'stretch_width')
automate_LL_aim = row(LL_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_LM_aim = row(LM_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_LR_aim = row(LR_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_RL_aim = row(RL_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_RM_aim = row(RM_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_RR_aim = row(RR_aim_slider, max_width = 400, sizing_mode = 'stretch_width')

game_stats_row_1 = row(game_stats_figure_1, game_stats_figure_2, 
                       max_width = 600, sizing_mode = "stretch_width")
game_stats_row_2 = row(game_stats_figure_3,
                       max_width = 600, sizing_mode = "stretch_width")
gui_column1 = column(game_figure, game_stats_row_1, game_stats_row_2,
                     buttons_bot, max_width = 600, sizing_mode = 'stretch_width')
gui_column2 = column(automate_button, strategy_dropdown_row, start_automate_row,
                     automate_LL_aim, automate_LM_aim, automate_LR_aim,
                     automate_RL_aim, automate_RM_aim, automate_RR_aim,
                     automation_table, automation_distribution_table, 
                     min_width = 761, max_width = 761, sizing_mode = 'stretch_width')

gui_row = row(gui_column1, gui_column2, max_width = 1400, sizing_mode = 'stretch_width')

grid1 = gridplot([[gui_row]], plot_width = 1200, plot_height = 480)

show(grid1)

### Create HTML File

In [ ]:
output_file("penalty_kick_game.html")
save(grid1)